In [1]:
import torch
import numpy as np
from neural_interaction_detection import get_interactions
from multilayer_perceptron import MLP, train, get_weights
from utils import (
    preprocess_data,
    get_pairwise_auc,
    get_anyorder_R_precision,
    set_seed,
    print_rankings,
)

In [32]:
use_main_effect_nets = True  # toggle this to use "main effect" nets
num_samples = 1000
num_features = 100

## Generate synthetic data with ground truth interactions

In [42]:
import numpy as np

BLOCK_DIM = 25
NUM_BLOCKS = 4
TOTAL_DIM = BLOCK_DIM * NUM_BLOCKS

# ---- Local benchmark functions (no py-benchmark dependency) ----
# All functions accept X with shape (N, dim) and return (N,)

def _sphere(X):
    return np.sum(X ** 2, axis=1)


def _rastrigin(X):
    n = X.shape[1]
    return 10.0 * n + np.sum(X ** 2 - 10.0 * np.cos(2.0 * np.pi * X), axis=1)


def _ackley(X):
    n = X.shape[1]
    s1 = np.sum(X ** 2, axis=1)
    s2 = np.sum(np.cos(2.0 * np.pi * X), axis=1)
    term1 = -20.0 * np.exp(-0.2 * np.sqrt(s1 / n))
    term2 = -np.exp(s2 / n)
    return term1 + term2 + 20.0 + np.e


def _rosenbrock(X):
    xi = X[:, :-1]
    xnext = X[:, 1:]
    return np.sum(100.0 * (xnext - xi ** 2) ** 2 + (xi - 1.0) ** 2, axis=1)


def _griewank(X):
    n = X.shape[1]
    sum_term = np.sum(X ** 2, axis=1) / 4000.0
    i = np.arange(1, n + 1)
    prod_term = np.prod(np.cos(X / np.sqrt(i)), axis=1)
    return sum_term - prod_term + 1.0


def _schwefel(X):
    n = X.shape[1]
    return 418.9829 * n - np.sum(X * np.sin(np.sqrt(np.abs(X))), axis=1)


def _levy(X):
    w = 1 + (X - 1) / 4
    term1 = np.sin(np.pi * w[:, 0]) ** 2
    term3 = (w[:, -1] - 1) ** 2 * (1 + np.sin(2 * np.pi * w[:, -1]) ** 2)
    wi = w[:, :-1]
    term2 = np.sum((wi - 1) ** 2 * (1 + 10 * np.sin(np.pi * wi + 1) ** 2), axis=1)
    return term1 + term2 + term3


def _zakharov(X):
    i = np.arange(1, X.shape[1] + 1)
    sum1 = np.sum(X ** 2, axis=1)
    sum2 = np.sum(0.5 * i * X, axis=1)
    return sum1 + sum2 ** 2 + sum2 ** 4


def _michalewicz(X, m=10):
    i = np.arange(1, X.shape[1] + 1)
    return -np.sum(np.sin(X) * (np.sin(i * X ** 2 / np.pi) ** (2 * m)), axis=1)


def _dixon_price(X):
    x1 = X[:, 0]
    i = np.arange(2, X.shape[1] + 1)
    xi = X[:, 1:]
    term1 = (x1 - 1) ** 2
    term2 = np.sum(i * (2 * xi ** 2 - X[:, :-1]) ** 2, axis=1)
    return term1 + term2


def _sum_squares(X):
    i = np.arange(1, X.shape[1] + 1)
    return np.sum(i * X ** 2, axis=1)


def _bent_cigar(X):
    return X[:, 0] ** 2 + 1e6 * np.sum(X[:, 1:] ** 2, axis=1)


def _discus(X):
    return 1e6 * X[:, 0] ** 2 + np.sum(X[:, 1:] ** 2, axis=1)


def _weierstrass(X, a=0.5, b=3.0, k_max=20):
    i = np.arange(0, k_max + 1)
    term1 = np.sum((a ** i) * np.cos(2 * np.pi * (b ** i) * (X[:, :, None] + 0.5)), axis=2)
    term1 = np.sum(term1, axis=1)
    term2 = X.shape[1] * np.sum((a ** i) * np.cos(2 * np.pi * (b ** i) * 0.5))
    return term1 - term2


def _ellipsoid(X):
    i = np.arange(1, X.shape[1] + 1)
    return np.sum((1e6 ** ((i - 1) / (X.shape[1] - 1))) * X ** 2, axis=1)


def _alpine1(X):
    return np.sum(np.abs(X * np.sin(X) + 0.1 * X), axis=1)


def _alpine2(X):
    return np.prod(np.sqrt(np.abs(X)) * np.sin(X), axis=1) * -1.0


def _katsuura(X):
    n = X.shape[1]
    i = np.arange(1, n + 1)
    j = np.arange(1, 33)
    term = np.sum(np.abs(2 ** j * X[:, :, None] - np.round(2 ** j * X[:, :, None])) / 2 ** j, axis=2)
    prod = np.prod((1 + i * term) ** (10 / n ** 1.2), axis=1)
    return prod - 1


def _salomon(X):
    r = np.sqrt(np.sum(X ** 2, axis=1))
    return 1 - np.cos(2 * np.pi * r) + 0.1 * r


def _whitley(X):
    n = X.shape[1]
    total = np.zeros(X.shape[0])
    for i in range(n):
        xi = X[:, i][:, None]
        for j in range(n):
            xj = X[:, j][:, None]
            temp = 100 * (xi ** 2 - xj) ** 2 + (1 - xj) ** 2
            total += (temp ** 2 / 4000.0 - np.cos(temp) + 1).reshape(-1)
    return total


def _bohachevsky(X):
    x1 = X[:, 0]
    x2 = X[:, 1]
    return x1 ** 2 + 2 * x2 ** 2 - 0.3 * np.cos(3 * np.pi * x1) - 0.4 * np.cos(4 * np.pi * x2) + 0.7


def _perm(X, beta=10):
    n = X.shape[1]
    j = np.arange(1, n + 1)
    total = np.zeros(X.shape[0])
    for i in range(1, n + 1):
        term = np.sum((j ** i + beta) * ((X / j) ** i - 1), axis=1)
        total += term ** 2
    return total


def _trid(X):
    term1 = np.sum((X - 1) ** 2, axis=1)
    term2 = np.sum(X[:, 1:] * X[:, :-1], axis=1)
    return term1 - term2


def _powell(X):
    # assumes dim is multiple of 4; for 25D, we wrap last block
    Xp = X.copy()
    if Xp.shape[1] % 4 != 0:
        pad = 4 - (Xp.shape[1] % 4)
        Xp = np.pad(Xp, ((0, 0), (0, pad)), mode="wrap")
    total = np.zeros(Xp.shape[0])
    for i in range(0, Xp.shape[1], 4):
        x1, x2, x3, x4 = Xp[:, i], Xp[:, i + 1], Xp[:, i + 2], Xp[:, i + 3]
        total += (x1 + 10 * x2) ** 2 + 5 * (x3 - x4) ** 2 + (x2 - 2 * x3) ** 4 + 10 * (x1 - x4) ** 4
    return total


def _styblinski_tang(X):
    return 0.5 * np.sum(X ** 4 - 16 * X ** 2 + 5 * X, axis=1)


def _quartic(X):
    i = np.arange(1, X.shape[1] + 1)
    return np.sum(i * X ** 4, axis=1)


def _brown(X):
    xi = X[:, :-1]
    xnext = X[:, 1:]
    return np.sum((xi ** 2) ** (xnext ** 2 + 1) + (xnext ** 2) ** (xi ** 2 + 1), axis=1)


def _ridge(X):
    return X[:, 0] + 100 * np.sqrt(np.sum(X[:, 1:] ** 2, axis=1))


FUNCTIONS = {
    "sphere": _sphere,
    "rastrigin": _rastrigin,
    "ackley": _ackley,
    "rosenbrock": _rosenbrock,
    "griewank": _griewank,
    "schwefel": _schwefel,
    "levy": _levy,
    "zakharov": _zakharov,
    "michalewicz": _michalewicz,
    "dixon_price": _dixon_price,
    "sum_squares": _sum_squares,
    "bent_cigar": _bent_cigar,
    "discus": _discus,
    "weierstrass": _weierstrass,
    "ellipsoid": _ellipsoid,
    "alpine1": _alpine1,
    "alpine2": _alpine2,
    "katsuura": _katsuura,
    "salomon": _salomon,
    "whitley": _whitley,
    "bohachevsky": _bohachevsky,
    "perm": _perm,
    "trid": _trid,
    "powell": _powell,
    "styblinski_tang": _styblinski_tang,
    "quartic": _quartic,
    "brown": _brown,
    "ridge": _ridge,
}

# Choose 25 names for the pool
POOL_NAMES = [
    "sphere",
    "rastrigin",
    "ackley",
    "rosenbrock",
    "griewank",
    "schwefel",
    "levy",
    "zakharov",
    "michalewicz",
    "dixon_price",
    "sum_squares",
    "bent_cigar",
    "discus",
    "weierstrass",
    "ellipsoid",
    "alpine1",
    "alpine2",
    "katsuura",
    "salomon",
    "whitley",
    "bohachevsky",
    "perm",
    "trid",
    "powell",
    "styblinski_tang",
]


def _random_rotation_matrix(dim, rng):
    H = rng.normal(size=(dim, dim))
    Q, _ = np.linalg.qr(H)
    if np.linalg.det(Q) < 0:
        Q[:, 0] *= -1
    return Q


def _resolve_shift(shift, rng, dim, scale):
    if shift is None or shift is False:
        return None
    if isinstance(shift, str):
        if shift == "random":
            return rng.uniform(-scale, scale, size=dim)
        raise ValueError(f"Unknown shift spec: {shift}")
    shift = np.asarray(shift, dtype=float)
    if shift.shape != (dim,):
        raise ValueError(f"shift must be shape ({dim},), got {shift.shape}")
    return shift


def _resolve_rotation(rotate, rng, dim):
    if rotate is None or rotate is False:
        return None
    if rotate is True or rotate == "random":
        return _random_rotation_matrix(dim, rng)
    rotate = np.asarray(rotate, dtype=float)
    if rotate.shape != (dim, dim):
        raise ValueError(f"rotation must be shape ({dim}, {dim}), got {rotate.shape}")
    return rotate


def _apply_transform(X, shift=None, rotation=None):
    X_t = X
    if shift is not None:
        X_t = X_t - shift
    if rotation is not None:
        X_t = X_t @ rotation
    return X_t


def make_pool_specs(names, rotate=True, shift="random", seed_base=0, weight=1.0, interaction=True):
    specs = []
    for i, name in enumerate(names):
        if name not in FUNCTIONS:
            raise ValueError(f"Unknown function name: {name}")
        specs.append(
            {
                "name": name,
                "rotate": rotate,
                "shift": shift,
                "seed": seed_base + i,
                "weight": weight,
                "interaction": interaction,
            }
        )
    return specs


def build_benchmark_pool(specs, dim, seed=123, shift_scale=1.0):
    rng_master = np.random.default_rng(seed)

    pool = []
    for spec in specs:
        spec = dict(spec)
        rng = np.random.default_rng(spec.get("seed", rng_master.integers(0, 2**32 - 1)))
        shift = _resolve_shift(spec.get("shift", None), rng, dim, shift_scale)
        rotation = _resolve_rotation(spec.get("rotate", False), rng, dim)

        fn = FUNCTIONS[spec["name"]]
        pool.append(
            {
                "name": spec["name"],
                "fn": fn,
                "shift": shift,
                "rotation": rotation,
                "weight": spec.get("weight", 1.0),
                "interaction": spec.get("interaction", True),
            }
        )

    return pool


def indices_from_names(pool, names):
    name_to_index = {spec["name"]: i for i, spec in enumerate(pool)}
    missing = [n for n in names if n not in name_to_index]
    if missing:
        raise ValueError(f"Names not in pool: {missing}")
    return [name_to_index[n] for n in names]


def make_composite_objective(pool, selected_indices, block_dim=BLOCK_DIM, num_blocks=NUM_BLOCKS):
    selected_indices = list(selected_indices)
    if len(selected_indices) != num_blocks:
        raise ValueError(f"selected_indices must be length {num_blocks}")

    def _objective(X):
        X = np.asarray(X)
        if X.ndim != 2 or X.shape[1] != block_dim * num_blocks:
            raise ValueError(f"X must be (N, {block_dim * num_blocks}). Got {X.shape}")

        Y = np.zeros(X.shape[0], dtype=float)
        ground_truth = []
        for block_i, pool_i in enumerate(selected_indices):
            spec = pool[pool_i]
            X_block = X[:, block_i * block_dim : (block_i + 1) * block_dim]
            X_block = _apply_transform(X_block, spec["shift"], spec["rotation"])
            Y = Y + spec["weight"] * spec["fn"](X_block)

            if spec.get("interaction", True):
                start = block_i * block_dim + 1
                end = (block_i + 1) * block_dim + 1
                ground_truth.append(set(range(start, end)))

        return Y, ground_truth

    return _objective


# ---- User-configurable pool and selection ----
POOL_SPECS = make_pool_specs(POOL_NAMES, rotate=True, shift="random", seed_base=0)
BENCHMARK_POOL = build_benchmark_pool(POOL_SPECS, dim=BLOCK_DIM, seed=123, shift_scale=1.0)

# Select 4 functions (each 25D) to compose a 100D objective.
# Option A: select by indices
SELECTED_POOL_INDICES = [0, 1, 2, 3]

# Option B: select by names (uncomment if you prefer names and have unique names)
# SELECTED_POOL_NAMES = ["sphere", "rastrigin", "ackley", "rosenbrock"]
# SELECTED_POOL_INDICES = indices_from_names(BENCHMARK_POOL, SELECTED_POOL_NAMES)

synth_func = make_composite_objective(BENCHMARK_POOL, SELECTED_POOL_INDICES)


In [43]:
set_seed(42)
X = np.random.uniform(low=-1, high=1, size=(num_samples, num_features))
Y, ground_truth = synth_func(X)
data_loaders = preprocess_data(
    X, Y, valid_size=100, test_size=100, std_scale=True, get_torch_loaders=True
)

## Train a multilayer perceptron (MLP)

In [44]:
device = torch.device("cpu")
model = MLP(
    num_features, [140, 100, 60, 20], use_main_effect_nets=use_main_effect_nets
).to(device)

In [45]:
import time

train_start = time.perf_counter()
model, mlp_loss = train(
    model, data_loaders, device=device, learning_rate=1e-2, l1_const=5e-5, verbose=True
)
train_seconds = time.perf_counter() - train_start


starting to train
early stopping enabled
[epoch 1, total 100] train loss: 3.4727, val loss: 1.0503
[epoch 3, total 100] train loss: 1.1290, val loss: 0.9284
[epoch 5, total 100] train loss: 0.9798, val loss: 0.8819
[epoch 7, total 100] train loss: 0.8218, val loss: 0.7768
[epoch 9, total 100] train loss: 0.5839, val loss: 0.4960
[epoch 11, total 100] train loss: 0.1865, val loss: 0.1349
[epoch 13, total 100] train loss: 0.0822, val loss: 0.0891
[epoch 15, total 100] train loss: 0.0501, val loss: 0.0641
[epoch 17, total 100] train loss: 0.0296, val loss: 0.0449
[epoch 19, total 100] train loss: 0.0201, val loss: 0.0311
[epoch 21, total 100] train loss: 0.0148, val loss: 0.0258
[epoch 23, total 100] train loss: 0.0128, val loss: 0.0224
[epoch 25, total 100] train loss: 0.0100, val loss: 0.0190
[epoch 27, total 100] train loss: 0.0105, val loss: 0.0153
[epoch 29, total 100] train loss: 0.0076, val loss: 0.0131
[epoch 31, total 100] train loss: 0.0067, val loss: 0.0137
[epoch 33, total 100

## Get the MLP's learned weights

In [46]:
model_weights = get_weights(model)

## Detect interactions from the weights

In [47]:
anyorder_interactions = get_interactions(model_weights, one_indexed=True)
pairwise_interactions = get_interactions(model_weights, pairwise=True, one_indexed=True)


print_rankings(pairwise_interactions, anyorder_interactions, top_k=10, spacing=14)

Pairwise interactions              Arbitrary-order interactions
(np.int64(16), np.int64(84))0.0000                      (np.int64(15), np.int64(18))0.0000        
(np.int64(6), np.int64(16))0.0000                      (np.int64(15), np.int64(18), np.int64(92))0.0000        
(np.int64(15), np.int64(29))0.0000                      (np.int64(6), np.int64(32))0.0000        
(np.int64(6), np.int64(32))0.0000                      (np.int64(15), np.int64(18), np.int64(76), np.int64(92))0.0000        
(np.int64(5), np.int64(18))0.0000                      (np.int64(6), np.int64(32), np.int64(37))0.0000        
(np.int64(6), np.int64(29))0.0000                      (np.int64(15), np.int64(18), np.int64(35), np.int64(76), np.int64(92))0.0000        
(np.int64(5), np.int64(29))0.0000                      (np.int64(3), np.int64(6), np.int64(32), np.int64(37))0.0000        
(np.int64(29), np.int64(70))0.0000                      (np.int64(3), np.int64(6), np.int64(32), np.int64(37), np.int64(42))0.

## Evaluate the interactions

In [48]:
auc = get_pairwise_auc(pairwise_interactions, ground_truth)
r_prec = get_anyorder_R_precision(anyorder_interactions, ground_truth)

print("Pairwise AUC", auc, ", Any-order R-Precision", r_prec)

Pairwise AUC 0.5105806451612903 , Any-order R-Precision 0.0


In [ ]:
import os
import csv
import json
from datetime import datetime

result_dir = "result"
os.makedirs(result_dir, exist_ok=True)
result_path = os.path.join(result_dir, "benchmark_results.csv")

selected_names = [BENCHMARK_POOL[i]["name"] for i in SELECTED_POOL_INDICES]
selected_resolved_names = [BENCHMARK_POOL[i].get("resolved_name", BENCHMARK_POOL[i]["name"]) for i in SELECTED_POOL_INDICES]
record = {
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "selected_indices": json.dumps(SELECTED_POOL_INDICES),
    "selected_names": json.dumps(selected_names),
    "selected_resolved_names": json.dumps(selected_resolved_names),
    "train_seconds": train_seconds,
    "pairwise_auc": auc,
    "anyorder_r_precision": r_prec,
}

write_header = not os.path.exists(result_path)
with open(result_path, "a", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=record.keys())
    if write_header:
        writer.writeheader()
    writer.writerow(record)

print(f"Saved results to {result_path}")
record
